In [61]:
import pandas as pd
import sqlite3

file_path = r"C:\Users\段\Desktop\724\Renewable_Energy_Adoption.csv"
df = pd.read_csv(file_path)
df = df.head(240)
df.index = range(1, len(df) + 1)
pd.set_option('display.max_rows', 300)

def fill_gov_investment(dataframe):
    country_avg = {}
    for country in dataframe['Country'].unique():
        country_data = dataframe[dataframe['Country'] == country]
        avg_investment = country_data['Government Investment (Million USD)'].mean()
        country_avg[country] = avg_investment if pd.notna(avg_investment) else 0
    for idx, row in dataframe.iterrows():
        if pd.isna(row['Government Investment (Million USD)']):
            current_country = row['Country']
            dataframe.at[idx, 'Government Investment (Million USD)'] = country_avg[current_country]
    return dataframe

df = fill_gov_investment(df)

energy_cols = ['Solar', 'Wind', 'Hydro', 'Geothermal', 'Biomass', 'Other Renewables']
df[energy_cols] = df[energy_cols].apply(pd.to_numeric, errors='coerce')
df['Renewable Energy (%)'] = pd.to_numeric(df['Renewable Energy (%)'], errors='coerce')

def compute_country_energy_avg(dataframe, cols):
    avg_dict = {}
    for country in dataframe['Country'].unique():
        for col in cols:
            country_data = dataframe[dataframe['Country'] == country]
            avg_val = country_data[col].mean()
            avg_dict[(country, col)] = avg_val if pd.notna(avg_val) else 0
    return avg_dict

country_energy_avg = compute_country_energy_avg(df, energy_cols)

def fill_energy_values(row, cols, country_avg_dict):
    missing_cols = [col for col in cols if pd.isna(row[col])]
    num_missing = len(missing_cols)
    if num_missing == 1:
        missing_col = missing_cols[0]
        if pd.isna(row['Renewable Energy (%)']):
            total_known = row[[col for col in cols if pd.notna(row[col])]].sum()
            row['Renewable Energy (%)'] = total_known
        else:
            known_sum = row[[col for col in cols if col != missing_col and pd.notna(row[col])]].sum()
            row[missing_col] = row['Renewable Energy (%)'] - known_sum
    else:
        for col in cols:
            if pd.isna(row[col]):
                row[col] = country_avg_dict.get((row['Country'], col), 0)
        if pd.isna(row['Renewable Energy (%)']):
            row['Renewable Energy (%)'] = row[cols].astype(float).fillna(0).sum()
    return row

df = df.apply(lambda row: fill_energy_values(row, energy_cols, country_energy_avg), axis=1)
df[energy_cols] = df[energy_cols].fillna(0)

print(df.head(240))



          Country  Year  Total Energy Consumption (TWh)  Renewable Energy (%)  \
1             USA  2000                            2383              7.027076   
2             USA  2001                            2734              6.816378   
3             USA  2002                            1607              9.538258   
4             USA  2003                            3587              4.334616   
5             USA  2004                            2363              5.678810   
6             USA  2005                            2667              5.478015   
7             USA  2006                            3874             13.870154   
8             USA  2007                            2709              4.502549   
9             USA  2008                            3619              3.062048   
10            USA  2009                            4011              7.926514   
11            USA  2010                            2966             32.172057   
12            USA  2011     

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
file_path = r"C:\Users\段\Desktop\724\Renewable_Energy_Adoption.csv"
df = pd.read_csv(file_path).head(30)

conn = sqlite3.connect(':memory:')
df.to_sql('renewable', conn, if_exists='replace', index=False)

query_investment = """
SELECT Country, AVG([Government Investment (Million USD)]) AS avg_investment
FROM renewable
GROUP BY Country
"""
df_investment = pd.read_sql_query(query_investment, conn)
plt.figure(figsize=(10,6))
plt.bar(df_investment['Country'], df_investment['avg_investment'])
plt.title('Average Government Investment by Country')
plt.xlabel('Country')
plt.ylabel('Government Investment (Million USD)')
plt.show()

query_energy = """
SELECT Country,
       AVG(Solar) AS avg_solar,
       AVG(Wind) AS avg_wind,
       AVG(Hydro) AS avg_hydro,
       AVG(Geothermal) AS avg_geothermal,
       AVG(Biomass) AS avg_biomass,
       AVG([Other Renewables]) AS avg_other
FROM renewable
GROUP BY Country
"""
df_energy = pd.read_sql_query(query_energy, conn)
df_energy.set_index('Country', inplace=True)
df_energy.plot(kind='bar', stacked=True, figsize=(10,6))
plt.title('Average Renewable Energy Components by Country')
plt.xlabel('Country')
plt.ylabel('Average Value')
plt.show()

query_scatter = """
SELECT [Government Investment (Million USD)] as investment, [Renewable Energy (%)] as renewable_pct
FROM renewable
WHERE [Government Investment (Million USD)] IS NOT NULL
  AND [Renewable Energy (%)] IS NOT NULL
"""
df_scatter = pd.read_sql_query(query_scatter, conn)
plt.figure(figsize=(10,6))
plt.scatter(df_scatter['investment'], df_scatter['renewable_pct'])
plt.title('Government Investment vs Renewable Energy (%)')
plt.xlabel('Government Investment (Million USD)')
plt.ylabel('Renewable Energy (%)')
plt.show()
